In [13]:
from model import Denoising
import data_generator as dg
from data_generator import DenoisingDataset

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import time
import torch.optim as optim

In [14]:
batch_size = 128
cuda = torch.cuda.is_available()
n_epoch = 100
sigma = 25
lr = 1e-3
data_dir = './data/Train400'
save_dir = './result'

In [19]:
model = Denoising()

model.train()
criterion = nn.MSELoss(reduction = 'sum')
if cuda:
    model = model.cuda()

optimizer = optim.Adam(model.parameters(), lr=lr)
for epoch in range(n_epoch):
    x = dg.datagenerator(data_dir=data_dir).astype('float32')/255.0
    print(x.shape)
    x = torch.from_numpy(x.transpose((0, 1, 2, 3)))
    dataset = DenoisingDataset(x, sigma)
    loader = DataLoader(dataset=dataset, num_workers=4, drop_last=True, batch_size=batch_size, shuffle=True)
    epoch_loss = 0
    start_time = time.time()
    
    for cnt, batch_yx in enumerate(DLoader):
        optimizer.zero_grad()
        if cuda:
            batch_x, batch_y = batch_yx[1].cuda(), batch_yx[0].cuda()
        loss = criterion(model(batch_y), batch_x)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        if cnt%10 == 0:
            print('%4d %4d / %4d loss = %2.4f' % (epoch+1, n_count, xs.size(0)//batch_size, loss.item()/batch_size))
    
    elapsed_time = time.time() - start_time

torch.save(model, save_dir+'/model_1.pth')    

init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
^_^-training data finished-^_^
(0, 1)


ValueError: axes don't match array